In [22]:
import os
import json
import pandas as pd
import traceback

In [23]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv()

True

In [24]:
KEY=os.getenv("GOOGLE_API_KEY")

In [25]:
llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7d7c7f8f5310>, default_metadata=())

In [26]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain,SequentialChain
# from langchain.callbacks import get_openai_callback
import PyPDF2

In [29]:
RESPONSE_JSON={
    "1":{
        "mcq" : "multiple choice question",
        "options" : {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here"
        },
        "correct" : "correct answer"
    },
    "2":{
        "mcq" : "multiple choice question",
        "options" : {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here"
        },
        "correct" : "correct answer"
    },
    "3":{
        "mcq" : "multiple choice question",
        "options" : {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here"
        },
        "correct" : "correct answer"
    }
}

In [27]:
TEMPLATE="""
TEXT : {text}
You are an expert MCQ maker.Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be confirming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs

### RESPONSE_JSON
{response_json}
"""

In [30]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [31]:
quiz_chain = LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

/tmp/ipykernel_16231/4090386649.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain = LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)


In [32]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students. \
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity
if the quiz is not at par with the cognitive and analytical abilities of the students, \
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities 
Quiz_MCQs: 
{quiz}
 
Check from an expert English Writer of the above quiz: 
"""

In [33]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject","quiz"],
    template=TEMPLATE2
)

In [34]:
review_chain = LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [35]:
generate_evaluate_chain=SequentialChain(
    chains=[quiz_chain,review_chain],
    input_variables=["text","number","subject","tone","response_json"],
    output_variables=["quiz","review"],
    verbose=True
)

In [36]:
file_path='/home/unique/Dev/mcqgen/data.txt'

In [37]:
with open(file_path,'r') as file:
    TEXT=file.read()

In [38]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence. The synonym self-teaching computers was also used in this time period.

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes. In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells. Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data. Other researchers who have studied human cognitive systems contributed to the modern machine lear

In [39]:
# serialie the python dictionary into JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [41]:
NUMBER = 5
SUBJECT = "Machine Learning"
TONE = "Simple"

In [42]:
response=generate_evaluate_chain.invoke(
    {
        "text" : TEXT,
        "number" : NUMBER,
        "subject" : SUBJECT,
        "tone" : TONE,
        "response_json" : json.dumps(RESPONSE_JSON)
    }
)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

TEXT : The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence. The synonym self-teaching computers was also used in this time period.

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes. In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells. Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to commu

In [43]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence. The synonym self-teaching computers was also used in this time period.\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes. In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells. Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data. Other researchers who have studied human cognitive systems contributed to the modern

In [50]:
quiz=response.get("quiz")
quiz_cleaned = quiz.strip().removeprefix("```json").removesuffix("```")

In [51]:
json.loads(quiz_cleaned)

{'1': {'mcq': 'Who coined the term "machine learning"?',
  'options': {'a': 'Donald Hebb',
   'b': 'Alan Turing',
   'c': 'Arthur Samuel',
   'd': 'Tom M. Mitchell'},
  'correct': 'c'},
 '2': {'mcq': 'What early device, developed by Raytheon, used reinforcement learning to analyze signals?',
  'options': {'a': 'A neural network',
   'b': 'Cybertron',
   'c': 'A checkers-playing program',
   'd': 'A computer terminal'},
  'correct': 'b'},
 '3': {'mcq': 'Which book introduced a theoretical neural structure that influenced AI and machine learning?',
  'options': {'a': "Nilsson's Learning Machines",
   'b': "Duda and Hart's Pattern Recognition",
   'c': 'The Organization of Behavior by Donald Hebb',
   'd': 'Computing Machinery and Intelligence by Alan Turing'},
  'correct': 'c'},
 '4': {'mcq': "According to Tom M. Mitchell's definition, what improves with experience in machine learning?",
  'options': {'a': 'The complexity of the algorithm',
   'b': 'The size of the dataset',
   'c': 'Per

In [54]:
quiz=json.loads(quiz_cleaned)
quiz_table_data=[]
for key,value in quiz.items():
    mcq=value["mcq"]
    options=" | ".join(
        [
            f"{option} : {option_value}"
            for option,option_value in value["options"].items()
        ]
    )
    correct=value["correct"]
    quiz_table_data.append({"MCQ":mcq,"Choices":options,"Correct":correct})


In [55]:
quiz_table_data

[{'MCQ': 'Who coined the term "machine learning"?',
  'Choices': 'a : Donald Hebb | b : Alan Turing | c : Arthur Samuel | d : Tom M. Mitchell',
  'Correct': 'c'},
 {'MCQ': 'What early device, developed by Raytheon, used reinforcement learning to analyze signals?',
  'Choices': 'a : A neural network | b : Cybertron | c : A checkers-playing program | d : A computer terminal',
  'Correct': 'b'},
 {'MCQ': 'Which book introduced a theoretical neural structure that influenced AI and machine learning?',
  'Choices': "a : Nilsson's Learning Machines | b : Duda and Hart's Pattern Recognition | c : The Organization of Behavior by Donald Hebb | d : Computing Machinery and Intelligence by Alan Turing",
  'Correct': 'c'},
 {'MCQ': "According to Tom M. Mitchell's definition, what improves with experience in machine learning?",
  'Choices': 'a : The complexity of the algorithm | b : The size of the dataset | c : Performance at tasks | d : The number of artificial neurons',
  'Correct': 'c'},
 {'MCQ':

In [57]:
quiz=pd.DataFrame(quiz_table_data)

In [58]:
quiz.to_csv("machinelearning.csv",index=False)

In [60]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'03_05_2025_20_04_00'